## This Script is used to get the unique sequences from GraphAligner.

In [1]:
# import the libraries
import numpy as np
import pandas as pd
import re 
from Bio import SeqIO
from collections import defaultdict
import gfapy

In [2]:
# Open the GFA file
file_path = "graph1.gfa"
gfa = gfapy.Gfa.from_file(file_path)

num_segments = len(gfa.segments)

In [3]:
data_graph = []

In [4]:
# Store the name and sequence for each node from the graph
for segment in gfa.segments:
    data_graph.append({"Name": segment.name, "Sequence": segment.sequence})

# Convert the list to a DataFrame
df_graph = pd.DataFrame(data_graph)

# Print the DataFrame
print(df_graph)

       Name                                           Sequence
0      1321  CGTTCCACCGGTTCTTACAGCCTGGTTACTCAGCAGCCGCTGGGTG...
1      1323  GTGCCTTACCACCCAGCGGCTGCTGAGTAACCAGGCTGTAAGAACC...
2      1325  GTGCCTTACCACCCAGCGGCTGCTGAGTAACCAGGCTGTAAGAACC...
3     32989  CTTAATATGAACCATCCAACTTTATGGGGTCAGTCCAGCAGCGCCG...
4       565  GGTTCGGCGGAGCTTACCGCGTCTTTTCGCGGTTAGCGGAGTGTGG...
...     ...                                                ...
2524  37173  GAACAAGGATCTAAGCTGTTTTAAGTTATGGGCAACGCAATGCACT...
2525  24893  TCTTAAGAGAGTGCATTGCGTTGCCCATAACTTAAAACAGCTTAGA...
2526  36779  TTTTCTCTGCAACCGAACCGGCTGTTTGTGTGAAGTGATTCACATC...
2527   6673  CTCGGCCCGACCCGAAGCCTGCAGGGATAAGTCGAAGGGACCGCGC...
2528  37823  CTCGGCCCGACCCGAAGCCTGCAGGGATAAGTCGAAGGGACCGCGC...

[2529 rows x 2 columns]


In [5]:
df_graph['Sequence']

0       CGTTCCACCGGTTCTTACAGCCTGGTTACTCAGCAGCCGCTGGGTG...
1       GTGCCTTACCACCCAGCGGCTGCTGAGTAACCAGGCTGTAAGAACC...
2       GTGCCTTACCACCCAGCGGCTGCTGAGTAACCAGGCTGTAAGAACC...
3       CTTAATATGAACCATCCAACTTTATGGGGTCAGTCCAGCAGCGCCG...
4       GGTTCGGCGGAGCTTACCGCGTCTTTTCGCGGTTAGCGGAGTGTGG...
                              ...                        
2524    GAACAAGGATCTAAGCTGTTTTAAGTTATGGGCAACGCAATGCACT...
2525    TCTTAAGAGAGTGCATTGCGTTGCCCATAACTTAAAACAGCTTAGA...
2526    TTTTCTCTGCAACCGAACCGGCTGTTTGTGTGAAGTGATTCACATC...
2527    CTCGGCCCGACCCGAAGCCTGCAGGGATAAGTCGAAGGGACCGCGC...
2528    CTCGGCCCGACCCGAAGCCTGCAGGGATAAGTCGAAGGGACCGCGC...
Name: Sequence, Length: 2529, dtype: object

In [6]:
#Save only one copy of each row to the dataframe
df_graph=df_graph.drop_duplicates()

In [7]:
#GraphAlignerNotEqual.tsv is from another script:GraphAlignerBandageScript.ipynb
df_GraphAligner_Paths= pd.read_csv('111_GraphResults/SequencesBandageGraphAligner.tsv', sep='\t')

In [ ]:
df_GraphAligner_Paths

In [ ]:
selected_rows = df_GraphAligner_Paths[df_GraphAligner_Paths['FinalResultBandageVSGraphAligner'] != 'Full']
selected_rows.columns

In [ ]:
selected_rows=selected_rows.drop(['Unnamed: 9',
       'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12', 'Unnamed: 13',
       'Unnamed: 14', 'Unnamed: 15','FinalResultBandageVSGraphAligner.1',
       'Unnamed: 17', 'Unnamed: 18', 'Unnamed: 19', 'Unnamed: 20',
       'Unnamed: 21', 'Unnamed: 22'] ,axis=1)


In [ ]:
selected_rows_Path

In [ ]:
# Function to process the Path values
def process_path(path):
    path = path.replace('<', ' ').replace('>', ' ')
    path = ' '.join(path.split('<'))
    return path

In [ ]:
df_GraphAligner_Paths['Path Matching']=df_GraphAligner_Paths['Path Matching'].apply(process_path).str.strip()

In [ ]:
df_GraphAligner_Paths[['Path1', 'Path2']] = df_GraphAligner_Paths['Path Matching'].str.split(' ', expand=True)

In [ ]:
df_GraphAligner_Paths

## Get the rows that have a unique node, Path1

In [ ]:
# Get the rows where Path 2 is  None
Path1_Only = df_GraphAligner_Paths[df_GraphAligner_Paths['Path2'].isna()]

# Print the resulting dataframe
Path1_Only 

In [ ]:
# Get the sequence where we have only Path1
merged_df_Path1= pd.merge(Path1_Only , df_graph, left_on='Path1', right_on='Name')
merged_df_Path1

In [ ]:
def extract_sequences_Path1(dataframe):
    for index, row in dataframe.iterrows():
        start_pos = row['Start Position on Path']
        end_pos = row['End Position on Path']
        sequence = row['Sequence']
        
        extracted_sequence = sequence[start_pos-1:end_pos]
        dataframe.loc[index, 'Result_node1'] = extracted_sequence
    
    return dataframe

In [ ]:
extracted_sequences_Path1_df = extract_sequences_Path1(merged_df_Path1)
extracted_sequences_Path1_df
#Save the dataframe to a pdf
extracted_sequences_Path1_df.to_csv("Path1_TestSequences.tsv", sep="\t")

In [ ]:
# Get the rows where Path 2 is not None
df_multiple = df_GraphAligner_Paths[df_GraphAligner_Paths['Path2'].notna()]

# Print the resulting dataframe
df_multiple

In [ ]:
merged_df_multiple_Path1= pd.merge(df_multiple, df_graph, left_on='Path1', right_on='Name')
merged_df_multiple_Path2=pd.merge(merged_df_multiple_Path1, df_graph, left_on='Path2', right_on='Name')
merged_df_multiple_Path2

In [ ]:
#Rename Name_X, Sequence_X, Name_y, Sequence_y
merged_df_multiple_Path2.rename(columns = {'Sequence_x':'Sequence_Path1'}, inplace = True)
merged_df_multiple_Path2.rename(columns = {'Sequence_y':'Sequence_Path2'}, inplace = True)
merged_df_multiple_Path2

In [ ]:
def extract_sequences_multiple_sequences(dataframe):
    for index, row in dataframe.iterrows():
        start_pos = row['Start Position on Path']
        sequence_1 = row['Sequence_Path1']
       
        #Get the first sequence, and remove 55 kmer from it
        extracted_sequence_first = sequence_1[start_pos-1:]
        
        dataframe.loc[index, 'Sequencefirst'] = extracted_sequence_first
    
    return dataframe

In [ ]:
extracted_sequences_2 = extract_sequences_multiple_sequences(merged_df_multiple_Path2)
extracted_sequences_2.to_csv('TestSeque.tsv',sep='\t')

extracted_sequences_2

In [ ]:
def extract_sequences_multiple_sequences_path2(dataframe):
    for index, row in dataframe.iterrows():
        end_pos = row['End Position on Path']
        sequence_2 = row['Sequence_Path2']
       
        #Get the first sequence, and remove 55 kmer from it
        extracted_sequence_second = sequence_2[:end_pos]
        
        dataframe.loc[index, 'SequencefSecond'] = extracted_sequence_second
    
    return dataframe

In [ ]:
extracted_sequences_3 = extract_sequences_multiple_sequences_path2(extracted_sequences_2)
extracted_sequences_3.to_csv('TestSeque3.tsv',sep='\t')

extracted_sequences_3

In [ ]:
def extract_sequences_multiple_sequences_overlap(dataframe):
   
    for index, row in dataframe.iterrows():
        sequence_first = row['Sequencefirst']
        length_end=len(sequence_first)-55
       
        #Get the first sequence, and the last 55 characters
        extracted_sequence_first = sequence_first[:len(sequence_first)-55]
        
        dataframe.loc[index, 'length'] = len(sequence_first)
        dataframe.loc[index, 'length_end'] = length_end
        dataframe.loc[index, 'FinalSequence1'] = sequence_first[:length_end]
    
    return dataframe
    

In [ ]:
extracted_sequences_multiple = extract_sequences_multiple_sequences_overlap(extracted_sequences_3)

In [ ]:
extracted_sequences_multiple

In [ ]:
def extract_sequences_multiple_sequences_final(dataframe):
   
    for index, row in dataframe.iterrows():
        sequence_first = row['FinalSequence1']
        sequence_second = row['SequencefSecond']
       
        #Get the first sequence, and the last 55 characters
        extracted_sequence_final = sequence_first+sequence_second
        
      
        dataframe.loc[index, 'FinalSequence'] = sequence_first + sequence_second
    
    return dataframe

In [ ]:
extracted_sequences_multiple_final =extract_sequences_multiple_sequences_final(extracted_sequences_multiple)

In [ ]:
extracted_sequences_multiple_final

In [ ]:
#Drop some columns
extracted_sequences_multiple_final=extracted_sequences_multiple_final.drop(['Path_Bandage', 'Path_GraphAligner','Equal','Name_x','Name_y'], axis=1)

In [ ]:
extracted_sequences_multiple_final

In [ ]:
extracted_sequences_multiple_final.rename(columns = {'Sequence_Path1':'Sequence_Path1'}, inplace = True)

In [ ]:
extracted_sequences_multiple_final.rename(columns = {'Sequence_Path2':'Sequence_FromGraph_Path2'}, inplace = True)

In [ ]:
extracted_sequences_multiple_final.rename(columns = {'Sequencefirst':'Sequence_FromGraph_Path1_No_Overlap'}, inplace = True)

In [ ]:
extracted_sequences_multiple_final.rename(columns = {'length':'Length_Sequence_Path1_No_Overlap'}, inplace = True)


In [ ]:
extracted_sequences_multiple_final.rename(columns = {'length_end':'Length_Sequence_Path1_With_Overlap'}, inplace = True)

In [ ]:
extracted_sequences_multiple_final.rename(columns = {'FinalSequence1':'Sequence_Path1_With_Overlap'}, inplace = True)

In [ ]:
extracted_sequences_multiple_final.rename(columns = {'FinalSequence':'Concatenated_Sequence'}, inplace = True)

In [ ]:
extracted_sequences_multiple_final.to_csv("FinalConcatenatedSequences.tsv",sep='\t')